In [24]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC,  LinearSVC
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.decomposition import PCA
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score, confusion_matrix, classification_report, precision_score
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.datasets import make_classification
import xgboost as xgb
from classification_model import xgb_classification, lgb_classification_2, svm_classification_2, calculate_precision
rus = RandomUnderSampler(random_state=2021)

In [25]:
df = []
files = os.listdir('../res2')
for i in files:
    if not i.endswith('txt'):
        df.append(pd.read_csv('../res2/'+i))
df = pd.concat(df)
df = df[df.columns[2:]]
df.shape

(26329, 431)

In [26]:
33 * 13 + 2 - 12

419

In [27]:
df.head()

,no,axis_x,axis_y,girth,girth_full,area,elongation,eccentricity,convexity,recty,...,hu_m4_11,hu_m5_11,hu_m6_11,contrast_11,dissimilarity_11,homogeneity_11,energy_11,correlation_11,ASM_11,dense_11
0,1001_35_-1,172.923065,124.384598,56.355338,57.355338,141.5,-2.157556,0.948526,1.160555,0.681296,...,-2.004229e-09,-2.402088e-06,2.032813e-09,1.049044,0.017520,0.999515,0.999496,0.599543,0.998992,365.189778
1,1001_35_-1,173.400009,124.800003,58.769552,62.769552,197.0,-1.580986,0.921891,1.168449,0.707615,...,-1.696064e-11,-1.037310e-06,3.092227e-09,1.026457,0.017920,0.999472,0.999449,0.615904,0.998899,363.281973
2,1090_35_-1,160.880005,131.339996,53.870057,56.870057,206.5,-0.681903,0.804048,1.216809,0.726908,...,-1.413873e-10,-6.336065e-07,-2.010870e-10,2.325441,0.039049,0.998971,0.998932,0.630963,0.997866,378.338183
3,1090_35_-1,162.000000,130.000000,58.870057,60.870057,239.5,-0.542389,0.761347,1.182848,0.748437,...,-1.510731e-10,-9.255876e-07,-1.027704e-09,2.602988,0.039650,0.998993,0.998961,0.616300,0.997923,375.634038
4,1091_35_-1,155.280823,158.917801,114.154328,115.154328,818.5,-0.119862,0.450121,1.168843,0.723512,...,-2.565188e-11,5.323312e-08,-5.349832e-11,5.801988,0.081102,0.998390,0.998369,0.510353,0.996741,370.216455


In [28]:
def  calculate_shift(x):
    axis_cols = [('axis_x_{}'.format(i), 'axis_y_{}'.format(i)) for i in range(12)]
    shifts = []
    a, b = x['axis_x'], x['axis_y']
    p_a, p_b = a, b
    for i in axis_cols:
        a_hat, b_hat = x[i[0]], x[i[1]]
        shift = np.sqrt(np.power(a-a_hat, 2) + np.power(b-b_hat, 2))
        a, b = a_hat, b_hat
        shifts.append(shift)
    a_hat, b_hat = x[axis_cols[-1][0]], x[axis_cols[-1][1]]
    shift = np.sqrt(np.power(p_a-a_hat, 2) + np.power(p_b-b_hat, 2))
    sum = np.sum(shifts)
    shifts.append(shift)
    shifts.append(sum)
    shifts.append(x['id'])
    return shifts

In [29]:
# 计算位移
df['id'] = list(range(df.shape[0]))
res = df.apply(calculate_shift, axis=1)
shift_cols = ['shift_{}'.format(i) for i in range(12)]
shift_cols = shift_cols + ['shift_sum', 'distance', 'id']
shift = pd.DataFrame(data=list(res), columns=shift_cols)
X = pd.merge(df, shift, on='id')
X.drop(columns=['id'], inplace=True)

In [30]:
X[[i for i in X.columns if not i.startswith('axis')]].head()

,no,girth,girth_full,area,elongation,eccentricity,convexity,recty,roundness,major_axis,...,shift_4,shift_5,shift_6,shift_7,shift_8,shift_9,shift_10,shift_11,shift_sum,distance
0,1001_35_-1,56.355338,57.355338,141.5,-2.157556,0.948526,1.160555,0.681296,0.540529,24.563128,...,1.421513,1.581158,0.999980,0.316221,0.316237,0.999992,0.806214,0.412306,4.967586,9.690323
1,1001_35_-1,58.769552,62.769552,197.0,-1.580986,0.921891,1.168449,0.707615,0.628316,26.019028,...,0.999980,0.316221,0.316237,0.999992,0.806214,0.412306,0.670814,0.582803,4.153104,10.039340
2,1090_35_-1,53.870057,56.870057,206.5,-0.681903,0.804048,1.216809,0.726908,0.802348,21.446333,...,1.060660,0.500000,8.513782,0.335000,0.338663,0.857214,0.364281,0.297795,11.168884,17.567078
3,1090_35_-1,58.870057,60.870057,239.5,-0.542389,0.761347,1.182848,0.748437,0.812284,22.245207,...,8.513782,0.335000,0.338663,0.857214,0.364281,0.297795,0.492527,0.401686,10.183136,15.929362
4,1091_35_-1,114.154328,115.154328,818.5,-0.119862,0.450121,1.168843,0.723512,0.775654,35.840611,...,1.445862,0.353562,1.114765,0.683001,1.196618,1.404376,0.780883,1.581139,3.271400,12.007259


In [31]:
X.to_csv('res/final_13.csv')

In [32]:
origin = pd.read_csv('../data/origin.csv')
origin = origin[origin.columns[1:]]

In [33]:
a = origin[['no', 'len_trace']].drop_duplicates()['len_trace']
num = 0 
size = 13
step = 2
for i in a.index:
    # print(a[i])
    if a[i] >= size:
        num += (a[i] - size) // step + 1
num

26329

In [34]:
origin.head()

,no,win_no,start,end,parent,frame_no,axis_x,axis_y,girth,girth_full,...,hu_m6,contrast,dissimilarity,homogeneity,energy,correlation,ASM,label,dense,len_trace
0,96_21_-1,21,11,16,0_21_-1,11,385.500000,571.000000,29.656854,35.656854,...,3.815276e-11,3.053937,0.027140,0.999724,0.999723,0.273471,0.999446,0,307.035862,6
1,96_21_-1,21,11,16,0_21_-1,12,384.500000,569.000000,36.899495,37.899495,...,2.459672e-10,3.597688,0.029776,0.999718,0.999717,0.294037,0.999435,0,299.315254,6
2,96_21_-1,21,11,16,0_21_-1,13,385.000000,569.500000,35.727922,36.727922,...,2.901733e-08,3.748888,0.029114,0.999739,0.999739,0.251333,0.999478,0,297.527692,6
3,96_21_-1,21,11,16,0_21_-1,14,384.774353,570.969421,28.899495,29.899495,...,2.179500e-07,3.988327,0.025613,0.999816,0.999816,0.123756,0.999632,0,303.486928,6
4,96_21_-1,21,11,16,0_21_-1,15,385.300018,570.100037,30.727922,34.970562,...,-6.243646e-10,4.115033,0.028699,0.999764,0.999764,0.297897,0.999528,0,298.393304,6


In [35]:
33 * 43 + 2 - 42

1379

In [36]:
for i in df.columns:
    print(i)

no
axis_x
axis_y
girth
girth_full
area
elongation
eccentricity
convexity
recty
roundness
major_axis
minor_axis
cent_m0
cent_m1
cent_m2
cent_m3
cent_m4
cent_m5
cent_m6
hu_m0
hu_m1
hu_m2
hu_m3
hu_m4
hu_m5
hu_m6
contrast
dissimilarity
homogeneity
energy
correlation
ASM
label
dense
axis_x_0
axis_y_0
girth_0
girth_full_0
area_0
elongation_0
eccentricity_0
convexity_0
recty_0
roundness_0
major_axis_0
minor_axis_0
cent_m0_0
cent_m1_0
cent_m2_0
cent_m3_0
cent_m4_0
cent_m5_0
cent_m6_0
hu_m0_0
hu_m1_0
hu_m2_0
hu_m3_0
hu_m4_0
hu_m5_0
hu_m6_0
contrast_0
dissimilarity_0
homogeneity_0
energy_0
correlation_0
ASM_0
dense_0
axis_x_1
axis_y_1
girth_1
girth_full_1
area_1
elongation_1
eccentricity_1
convexity_1
recty_1
roundness_1
major_axis_1
minor_axis_1
cent_m0_1
cent_m1_1
cent_m2_1
cent_m3_1
cent_m4_1
cent_m5_1
cent_m6_1
hu_m0_1
hu_m1_1
hu_m2_1
hu_m3_1
hu_m4_1
hu_m5_1
hu_m6_1
contrast_1
dissimilarity_1
homogeneity_1
energy_1
correlation_1
ASM_1
dense_1
axis_x_2
axis_y_2
girth_2
girth_full_2
area_2
elo

In [37]:
pri

NameError: name 'pri' is not defined